# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.functions import asc
from pyspark.sql.functions import desc
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
import pyspark
from pyspark import SparkConf
# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [27]:
spark=SparkSession\
        .builder \
        .appName("challenge 1")\
        .getOrCreate()

In [28]:
spark.sparkContext.getConf().getAll() 

[('spark.app.name', 'challenge 1'),
 ('spark.driver.port', '35097'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '8104ce72d034'),
 ('spark.app.id', 'local-1658083252736'),
 ('spark.ui.showConsoleProgress', 'true')]

In [29]:
path="data/sparkify_log_small.json"
user_log=spark.read.json(path)

In [31]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [35]:
user_log.select('page').where(user_log['userId']=="").dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [37]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.filter(user_log['userId']=="").show()

+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|artist|      auth|firstName|gender|itemInSession|lastName|length|level|location|method| page|registration|sessionId|song|status|           ts|userAgent|userId|
+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|  null|Logged Out|     null|  null|            0|    null|  null| free|    null|   PUT|Login|        null|     5598|null|   307|1513721196284|     null|      |
|  null|Logged Out|     null|  null|           26|    null|  null| paid|    null|   GET| Home|        null|      428|null|   200|1513721274284|     null|      |
|  null|Logged Out|     null|  null|            5|    null|  null| free|    null|   GET| Home|        null|     2941|null|   200|1513722009284|     null|      |
|  null|Logged Out|     null|  nul

# Question 3

How many female users do we have in the data set?

In [38]:
user_log.head()

Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')

In [39]:
# TODO: write your code to answer question 3
user_log.filter(user_log['gender']=='F').count()

3820

# Question 4

How many songs were played from the most played artist?

In [46]:
# TODO: write your code to answer question 4
user_log.groupby('artist').sum().sort('artist',ascending=False).show()

+--------------------+------------------+------------------+-----------------+--------------+-----------+--------------+
|              artist|sum(itemInSession)|       sum(length)|sum(registration)|sum(sessionId)|sum(status)|       sum(ts)|
+--------------------+------------------+------------------+-----------------+--------------+-----------+--------------+
|   ÃÂlafur Arnalds|                14|         485.85098|    1504658577284|           485|        200| 1513774488284|
|     ÃÂtienne Daho|                12|         235.25832|    1511351741284|          2631|        200| 1513786513284|
|             tobyMac|                16| 611.4211300000001|    3016594136568|          3007|        400| 3027569789568|
|the bird and the bee|               115|2003.5355100000002|   13515129596556|         41278|       1800|13624140816556|
|taylor deupree + ...|                12|          385.2273|    1513381884284|          5162|        200| 1513791772284|
|                moe.|          

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [52]:
# TODO: write your code to answer question 5
user_log.groupby('userId').mean().show()

+------+------------------+------------------+-----------------+-----------------+-----------------+--------------------+
|userId|avg(itemInSession)|       avg(length)|avg(registration)|   avg(sessionId)|      avg(status)|             avg(ts)|
+------+------------------+------------------+-----------------+-----------------+-----------------+--------------------+
|  2904|               0.0|         348.57751|1.513494341284E12|           2903.0|            200.0|   1.513721095284E12|
|   691|               1.5|269.66158666666666|1.503986110284E12|            690.0|            200.0|   1.513744509534E12|
|  2294|32.306451612903224| 253.2148934545454|1.487337122284E12|           2293.0|203.4516129032258|1.513779889396903...|
|  2162|              19.0|243.81803382352942|1.503425779284E12|           4814.0|            200.0|1.513777202822461...|
|  1436|               0.5|        316.695055|1.505550349284E12|           1435.0|            200.0|   1.513783058784E12|
|  2088|               6